<a href="https://colab.research.google.com/github/diqnfl777/2022F-Ajou-ML-TEAM3/blob/main/IndexModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지 설치 및 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 트랜스포머 설치
!pip install transformers

In [2]:
from transformers import AutoTokenizer
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, tqdm_notebook


### 데이터 가공 파트


In [3]:
#데이터 가공
#청구기호 숫자 앞 2개만 따와서 각각 매핑. 65~70은 예외적으로 경영학
data = pd.read_csv("/content/gdrive/MyDrive/LibraryCsv/indexhapbon.csv",encoding='UTF8',usecols = ['제목', '청구번호','목차'])
data = data.loc[:,['제목', '청구번호','목차']]
data = data.dropna()
data = data.drop_duplicates()
data

,제목,청구번호,목차
0,어디서 무엇이 되어 다시 만나랴,811.3 최68어샘,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
1,나의 사랑 클레멘타인,811.3 최68나샘,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
2,초파리를 알면 유전자가 보인다,595.774 M699fK이,서문 \r\n 서론 \r\n제1장 지도\r\n제2장 ...
3,아웃사이더의 반란,320.5662 R518rK장,한국어판 서문 / 5\r\n 프롤로그 / 13\r\n제1장 보수...
4,브랜드 인문학 :잠재된 표현 욕망을 깨우는 감각 수업,658.827 김25브,"1부-정체성\r\n1 프라다: 나의 정체성을 알 때, 비로소 브랜드는 ‘필..."
...,...,...,...
71533,한국 시극사 연구,811.209 이52한,ㆍ책머리에 \r\n\r\n제1장 서론 \r\n\r\n1. 연구 목적…17 \r\n2...
71534,조선의 서정시인 퇴계 이황 : 우리가 몰랐던 퇴계의 남도여행,811.14 정66조,"ㆍ책머리에\r\n\r\n1. 퇴계의 시혼, 남도에서 피다\r\n퇴계가 남쪽으로 간 ..."
71535,"내면기행 : 선인들, 스스로 묘비명을 쓰다",929.5 심14내,ㆍ책을 엮으며\r\n\r\n제1부 이사람을 보라\r\n제2부 이것으로 만족이다\r\...
71536,뉴 스토리 뉴 스타일,070.4 송52뉴,ㆍ책을 펴내며\r\n\r\n서 장 새로움을 위해\r\n제1장 살인의 추억\r\n제2...


In [4]:
import re 
data['청구번호'] = data['청구번호'].replace({r'(.*?)(\d{2})\d.*' : r'\2'}, regex=True)
data

,제목,청구번호,목차
0,어디서 무엇이 되어 다시 만나랴,81,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
1,나의 사랑 클레멘타인,81,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
2,초파리를 알면 유전자가 보인다,59,서문 \r\n 서론 \r\n제1장 지도\r\n제2장 ...
3,아웃사이더의 반란,32,한국어판 서문 / 5\r\n 프롤로그 / 13\r\n제1장 보수...
4,브랜드 인문학 :잠재된 표현 욕망을 깨우는 감각 수업,65,"1부-정체성\r\n1 프라다: 나의 정체성을 알 때, 비로소 브랜드는 ‘필..."
...,...,...,...
71533,한국 시극사 연구,81,ㆍ책머리에 \r\n\r\n제1장 서론 \r\n\r\n1. 연구 목적…17 \r\n2...
71534,조선의 서정시인 퇴계 이황 : 우리가 몰랐던 퇴계의 남도여행,81,"ㆍ책머리에\r\n\r\n1. 퇴계의 시혼, 남도에서 피다\r\n퇴계가 남쪽으로 간 ..."
71535,"내면기행 : 선인들, 스스로 묘비명을 쓰다",92,ㆍ책을 엮으며\r\n\r\n제1부 이사람을 보라\r\n제2부 이것으로 만족이다\r\...
71536,뉴 스토리 뉴 스타일,07,ㆍ책을 펴내며\r\n\r\n서 장 새로움을 위해\r\n제1장 살인의 추억\r\n제2...


In [5]:
data['청구번호'] = pd.to_numeric(data['청구번호'],errors = 'coerce')
data = data.dropna()
data.loc[(data['청구번호'] >= 100), ['청구번호']] = None
data = data.dropna()
data.loc[(data['청구번호'] < 10), ['청구번호']] = 1000
data.loc[(data['청구번호'] < 20), ['청구번호']] = 1001
data.loc[(data['청구번호'] < 30), ['청구번호']] = 1002
data.loc[(data['청구번호'] < 40), ['청구번호']] = 1003
data.loc[(data['청구번호'] < 50), ['청구번호']] = 1004
data.loc[(data['청구번호'] < 60), ['청구번호']] = 1005
data.loc[(data['청구번호'] < 65), ['청구번호']] = 1006 #경영학쪽은 따로 분류이기 떄문에 65 사용
data.loc[(data['청구번호'] < 70), ['청구번호']] = 1007 #즉, 1007 쪽이 경영학책
data.loc[(data['청구번호'] < 80), ['청구번호']] = 1008
data.loc[(data['청구번호'] < 90), ['청구번호']] = 1009
data.loc[(data['청구번호'] <  100), ['청구번호']] = 1010

labels = {'0':'총류',
          '1':'철학',
          '2':'종교',
          '3':'사회학',
          '4':'언어',
          '5':'자연과학',
          '6':'기술과학',
          '7':'경영학',
          '8':'예술',
          '9':'문학',
          '10':'역사'
          }

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [6]:
data['청구번호'].value_counts()

1003.0    16104
1009.0    12101
1006.0     7304
1007.0     6982
1001.0     5512
1010.0     5298
1000.0     5227
1005.0     5113
1008.0     4231
1004.0     2162
1002.0     1444
Name: 청구번호, dtype: int64

In [7]:
data['청구번호'] = data['청구번호']%1000
data['청구번호'] = data['청구번호'].astype(int)


In [8]:
'''
data['청구번호'] = data['청구번호'].apply(str) #스트링 변환, 참고용. 모델 돌리려면 실행하면 안됩니다
data['청구번호'] = data['청구번호'].replace(labels)
data
'''

"\ndata['청구번호'] = data['청구번호'].apply(str) #스트링 변환, 참고용. 모델 돌리려면 실행하면 안됩니다\ndata['청구번호'] = data['청구번호'].replace(labels)\ndata\n"

###KOR


### ENG

In [9]:
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.75):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 11)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [10]:
from tokenizers import Tokenizer
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig


en_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class EN_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels =  df['청구번호'].values
        self.dic = [en_tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['목차']]
    
    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def __getitem__(self, idx):

        batch_texts = self.dic[idx]
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [11]:
data['청구번호'].value_counts()

3     16104
9     12101
6      7304
7      6982
1      5512
10     5298
0      5227
5      5113
8      4231
4      2162
2      1444
Name: 청구번호, dtype: int64

In [12]:
indice = []
for i in range(0,11):
  indice.append(data.index[data['청구번호']==i]) 
for i in range(0,11):
  data = data.drop(indice[i][int(1444):]) # get first 1138

data['청구번호'].value_counts()

9     1444
5     1444
3     1444
7     1444
0     1444
1     1444
4     1444
6     1444
8     1444
2     1444
10    1444
Name: 청구번호, dtype: int64

In [15]:
data_sample = data #.sample(frac = 0.1,random_state = 123)
np.random.seed(212)
df_train, df_val, df_test = np.split(data_sample.sample(frac=1, random_state=42), 
                                     [int(.8*len(data_sample)), int(.9*len(data_sample))])
print("train:", len(df_train), "val:",len(df_val),"test:", len(df_test))

train: 12707 val: 1588 test: 1589


In [16]:
from torch.optim import AdamW
from tqdm import tqdm
from transformers.optimization import get_cosine_schedule_with_warmup


def train(model, train_data, val_data, learning_rate, epochs):

    train, val = EN_Dataset(train_data), EN_Dataset(val_data)

    warmup_rate = 0.1
    batch_size=14
    num_total_steps = (len(df_train) // batch_size) * epochs
    num_warmup_steps = num_total_steps * warmup_rate

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True,num_workers=3)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr= learning_rate)
    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                         num_warmup_steps=num_warmup_steps, 
                                                         num_training_steps=num_total_steps)
    if use_cuda:
            print("using cuda!")
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for i, (train_input, train_label) in enumerate(tqdm(train_dataloader)) :

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
                scheduler.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}') 
                  
EPOCHS = 5
model = BertClassifier()
LR = 0.00001          
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


using cuda!


100%|██████████| 908/908 [23:28<00:00,  1.55s/it]


Epochs: 1 | Train Loss:  0.127                 | Train Accuracy:  0.415                 | Val Loss:  0.079                 | Val Accuracy:  0.679


100%|██████████| 908/908 [23:35<00:00,  1.56s/it]


...

Epochs: 3 | Train Loss:  0.040                 | Train Accuracy:  0.852                 | Val Loss:  0.059                 | Val Accuracy:  0.763


100%|██████████| 908/908 [23:36<00:00,  1.56s/it]


Epochs: 4 | Train Loss:  0.025                 | Train Accuracy:  0.918                 | Val Loss:  0.058                 | Val Accuracy:  0.764


100%|██████████| 908/908 [23:36<00:00,  1.56s/it]


Epochs: 5 | Train Loss:  0.019                 | Train Accuracy:  0.946                 | Val Loss:  0.059                 | Val Accuracy:  0.765


In [17]:
def evaluate(model, test_data):

    test = EN_Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.770


In [ ]:
df_test.sample(frac=0.02,random_state=3) #

,제목,청구번호
119599,The economics of competitive enterprise : sele...,3
402519,Themes in modern European history 1890-1945,10
59007,Indische geisteswelt,2
121554,Myanmar (Burma) since 1962 : the failure of de...,3
117384,Harpers Ferry armory and the new technology :t...,3
...,...,...
305505,Off balance : the real world of ballet,8
394256,Regional geography of the world,10
11584,Inside secrets visual C++ 5.0 tips & tricks,0
434150,Solar energy dictionary,6


In [ ]:
def get_top_values(model, text):
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
      model = model.cuda()
  with torch.no_grad():
    dic = en_tokenizer(text, padding='max_length', max_length = 256, truncation=True, return_tensors="pt")
    input_id = dic['input_ids']
    input_id = input_id.to(device)
    mask = dic['attention_mask']
    mask = mask.to(device)
    output = model(input_id, mask)
    output = F.softmax(output[0],dim=-1)
    print("max: ",torch.topk(output,3))
get_top_values(model, "history") # 확인해보고 싶은 제목 입력
labels

max:  torch.return_types.topk(
values=tensor([0.9592, 0.0095, 0.0089], device='cuda:0'),
indices=tensor([10,  9,  8], device='cuda:0'))


{'0': '총류',
 '1': '철학',
 '2': '종교',
 '3': '사회학',
 '4': '언어',
 '5': '자연과학',
 '6': '기술과학',
 '7': '경영학',
 '8': '예술',
 '9': '문학',
 '10': '역사'}

### 참고용

In [18]:
def save_checkpoint(save_path, model, valid_loss):# ------ 모델 평가를 위해 훈련 과정을 저장
    if save_path == None:
        return
    state_dict = {'model_state_dict': model.state_dict(), 'valid_loss': valid_loss}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):# ------ save_checkpoint 함수에서 저장된 모델을 가져오기.
    if load_path == None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [19]:
save_checkpoint("/content/gdrive/MyDrive/Learned/indexmodel1.multiLangModel", model, 1)

Model saved to ==> /content/gdrive/MyDrive/Learned/indexmodel1.multiLangModel


In [ ]:
device =torch.device("cuda")
load_checkpoint("/content/gdrive/MyDrive/Learned/0.1_loss1e-6_epoch5.multiLangModel",model)


Model loaded from <== /content/gdrive/MyDrive/Learned/0.1_loss1e-6_epoch5.multiLangModel


0.201

In [ ]:
class MyModelA(nn.Module):  # 이후 합칠때 참고용
    def __init__(self):
        super(MyModelA, self).__init__()
        self.fc1 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        return x
    

class MyModelB(nn.Module):
    def __init__(self):
        super(MyModelB, self).__init__()
        self.fc1 = nn.Linear(20, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        return x


class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = nn.Linear(4, 2)
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(F.relu(x))
        return x

# Create models and load state_dicts    
modelA = MyModelA()
modelB = MyModelB()
# Load state dicts
modelA.load_state_dict(torch.load(PATH))
modelB.load_state_dict(torch.load(PATH))

model = MyEnsemble(modelA, modelB)
x1, x2 = torch.randn(1, 10), torch.randn(1, 20)
output = model(x1, x2)